<a href="https://colab.research.google.com/github/Enzo-Damico/projet_MODIA/blob/master/ncf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/Projet/"

Mounted at /content/drive


In [32]:
import pandas as pd
import zipfile

#df_interactions_test = pd.read_csv(path + "interactions_test.csv")
#df_interactions_train = pd.read_csv(path + "interactions_train.csv.zip")
df_RAW_interactions = pd.read_csv(path + "RAW_interactions.csv.zip")
#df_RAW_recipes = pd.read_csv(path + "RAW_recipes.csv.zip")
print(df_RAW_interactions.shape)
#print(df_RAW_recipes.shape)

(1132367, 5)


In [3]:
import ast

recipes_nvote = df_RAW_interactions["recipe_id"].value_counts()
rating = df_RAW_interactions[["recipe_id", "rating"]].groupby(["recipe_id"]).mean()
df_RAW_recipes["vote_count"] = df_RAW_recipes["id"].apply(lambda x: recipes_nvote.loc[x])
df_RAW_recipes["vote_mean"] = df_RAW_recipes["id"].apply(lambda x: rating.loc[x])
df_RAW_recipes["ingredients"] = df_RAW_recipes["ingredients"].apply(lambda s: ast.literal_eval(s)) 
df_RAW_recipes["steps"] = df_RAW_recipes["steps"].apply(lambda s: s.strip('[]'))

In [4]:
m = 100
c = df_RAW_recipes.vote_mean.mean()

def weighted_score(x):
    v = x['vote_count']
    r = x['vote_mean']
    return (v/(v+m) * r) + (m/(m+v) * c)

df_RAW_recipes["vote_w"] = df_RAW_recipes[["vote_count", "vote_mean"]].apply(lambda x: weighted_score(x), axis=1)

In [5]:
subset = df_RAW_recipes[df_RAW_recipes['vote_count'] > 10].reset_index()

In [41]:
df_RAW_interactions

,user_id,recipe_id,date,rating,review,count
0,2445606,63828,2012-10-11,5,Fabulous!!!! Yummy.. The best ever... OMG... wow,1
1,28846,26110,2002-10-14,5,"Luscious, melt in your mouth, buttery biscuits...",4
2,20371,26110,2002-10-20,5,These biscuits were exceptional: tender and mo...,11
3,28649,26110,2002-10-27,5,This were a bit hit with my family. I used a ...,12
4,49937,26110,2002-12-08,5,These turned out fabulous! I used the food pro...,3
...,...,...,...,...,...,...
44116,2002077248,131018,2018-04-14,5,This was AMAZING. I had 2 3lb partially frozen...,1
44117,2000023731,131018,2018-06-08,5,I am a 'pulled pork' freak. I'm always looking...,1
44118,2002200471,131018,2018-06-27,0,No need to season pork at all?,1
44119,2002276261,131018,2018-09-16,5,Excellent! Made the famous Dave's BBQ sauce al...,1


In [33]:
# On garde les 100 recettes qui ont le plus de vote
df_RAW_interactions["count"] = df_RAW_interactions.groupby("recipe_id").transform('count')['user_id']
recipeId = df_RAW_interactions.drop_duplicates('recipe_id').sort_values(
    'count', ascending=False).iloc[:100]['recipe_id']
df_RAW_interactions = df_RAW_interactions[df_RAW_interactions['recipe_id'].isin(recipeId)].reset_index(drop=True)

# On garde les 100 utilisateurs qui ont le plus voté
df_RAW_interactions["count"] = df_RAW_interactions.groupby("user_id").transform('count')['recipe_id']
userId = df_RAW_interactions.drop_duplicates('user_id').sort_values(
    'count', ascending=False).iloc[:20001]['user_id']
df_RAW_interactions = df_RAW_interactions[df_RAW_interactions['user_id'].isin(userId)].reset_index(drop=True)

In [46]:
userId

256      140132
649     1581225
1008     218535
47       176615
280     1133190
         ...   
8876    2140832
8903    1635242
8898    1314352
8897    2262192
8896    2258596
Name: user_id, Length: 20001, dtype: int64

In [48]:
user2id

{2445606: 0,
 28846: 1,
 28649: 2,
 49937: 3,
 60575: 4,
 44160: 5,
 13063: 6,
 107651: 7,
 133621: 8,
 89015: 9,
 74028: 10,
 8688: 11,
 28346: 12,
 5060: 13,
 20238: 14,
 60694: 15,
 62043: 16,
 35526: 17,
 59992: 18,
 92598: 19,
 91867: 20,
 29212: 21,
 60260: 22,
 68588: 23,
 61357: 24,
 39835: 25,
 58413: 26,
 115525: 27,
 134624: 28,
 123897: 29,
 26075: 30,
 62191: 31,
 89240: 32,
 35701: 33,
 130663: 34,
 95743: 35,
 271221: 36,
 267665: 37,
 302094: 38,
 222433: 39,
 335606: 40,
 38937: 41,
 268272: 42,
 365242: 43,
 402559: 44,
 97579: 45,
 120121: 46,
 128473: 47,
 316194: 48,
 337011: 49,
 148715: 50,
 461834: 51,
 349821: 52,
 95241: 53,
 394617: 54,
 458309: 55,
 474474: 56,
 280166: 57,
 471673: 58,
 163986: 59,
 556250: 60,
 526505: 61,
 306469: 62,
 429064: 63,
 230892: 64,
 25438: 65,
 169166: 66,
 285011: 67,
 125808: 68,
 400420: 69,
 129652: 70,
 491970: 71,
 565910: 72,
 173579: 73,
 543953: 74,
 714655: 75,
 262272: 76,
 64203: 77,
 530731: 78,
 285041: 79,
 7475

In [34]:
testset = df_RAW_interactions.sample(frac=0.1, replace=False)
trainset = df_RAW_interactions[~df_RAW_interactions.index.isin(testset.index)]

In [35]:
import torch
from torch.utils.data import Dataset, DataLoader

user_list = trainset.user_id.unique()
item_list = trainset.recipe_id.unique()
user2id = {w: i for i, w in enumerate(user_list)}
item2id = {w: i for i, w in enumerate(item_list)}

class Ratings_Datset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index()

    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        user = user2id[self.df['user_id'][idx]]
        user = torch.tensor(user, dtype=torch.long)
        item = item2id[self.df['recipe_id'][idx]]
        item = torch.tensor(item, dtype=torch.long)
        rating = torch.tensor(self.df['rating'][idx], dtype=torch.float)
        return user, item, rating


trainloader = DataLoader(Ratings_Datset(trainset), batch_size=512, shuffle=True ,num_workers=2)
testloader = DataLoader(Ratings_Datset(testset), batch_size=64, num_workers=2)

In [36]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn

class NCF(nn.Module):
        
    def __init__(self, n_users, n_items, n_factors=8):
        super().__init__()
        self.user_embeddings = torch.nn.Embedding(n_users, n_factors)
        self.item_embeddings = torch.nn.Embedding(n_items, n_factors)
        self.predictor = torch.nn.Sequential(
            nn.Linear(in_features=n_factors*2, out_features=64),
            nn.Linear(in_features=64, out_features=32),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )
        
        
    def forward(self, user, item):
        

        u = self.user_embeddings(user)
        i = self.item_embeddings(item)

        # Concat the two embedding layers
        z = torch.cat([u, i], dim=-1)
        return self.predictor(z)

In [37]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from statistics import mean


def train(model, optimizer, trainloader, epochs=30):
    criterion = nn.MSELoss(reduction='mean')
    t = tqdm(range(epochs))
    for epoch in t:
        corrects = 0
        total = 0
        train_loss = []
        for users, items, r in trainloader:
            users = users.cuda()
            items = items.cuda()
            r = r.cuda() / 5
            y_hat = model(users, items)
            loss = criterion(y_hat, r.unsqueeze(1).float())
            train_loss.append(loss.item())
            total += r.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            t.set_description(f"loss: {mean(train_loss)}")

In [38]:
n_user = trainset.user_id.nunique()
n_items = trainset.recipe_id.nunique()
model = NCF(n_user, n_items).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train(model, optimizer, trainloader, epochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
import math

def test(model, testloader, m_eval=False):

    running_mae = 0
    with torch.no_grad():
        corrects = 0
        total = 0
        for users, items, r in testloader:
            users = users.cuda()
            items = items.cuda()
            y = r.cuda() / 5
            y_hat = model(users, items).flatten()
            error = torch.abs(y_hat - y).sum().data
            
            running_mae += error
            total += y.size(0)
    
    mae = running_mae/total
    return mae * 5

In [30]:
for key, value in user2id.items():
  if key == 363891:
    print('ok')

In [39]:
test(model, testloader)

KeyError: ignored